In [1]:
import pickle
import pandas as pd
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import sklearn

In [2]:
with open('data/pickle/vectors_neg.pkl', 'rb') as f:
    vecs_neg = np.array(pickle.load(f))
    
with open('data/pickle/vectors_pos.pkl', 'rb') as f:
    vecs_pos = np.array(pickle.load(f))

In [3]:
x = np.concatenate((vecs_neg, vecs_pos), axis = 0)
mean = np.mean(x, axis=0)
std = np.std(x, axis=0)
x = (x - mean) / std

In [4]:
x.shape

(200000, 768)

In [5]:
y = np.concatenate((np.zeros(len(vecs_neg)), np.ones(len(vecs_pos))))

In [6]:
y.shape

(200000,)

In [7]:
del vecs_neg
del vecs_pos

In [8]:
np.mean(x)

-3.632742255158897e-18

In [9]:
def train_and_pred(x_train, y_train, x_test):
    classifier = LinearSVC(verbose=True, max_iter = 5000)#RidgeClassifier()#RandomForestClassifier(n_jobs=-1, verbose = True)
    classifier.fit(x_train, y_train)

    return classifier.predict(x_test)

def crossValidate(x, y, fold):
    accuracy = 0
    kf = KFold(n_splits=fold, shuffle = True)
    
    for train_index, test_index in kf.split(y):
        train_x = x[train_index, :]
        train_y = y[train_index]
        test_x = x[test_index, :]
        test_y = y[test_index]
        
        pred = train_and_pred(train_x, train_y, test_x)
        
        accuracy += accuracy_score(test_y, pred, normalize=False)
        print(accuracy_score(test_y, pred))
        
    print("avg fold accuracy : ", accuracy / len(y))



In [ ]:
crossValidate(x, y, 5)

[LibLinear]....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
optimization finished, #iter = 5000

Using -s 2 may be faster (also see FAQ)

Objective value = -80267.494533
nSV = 119641
0.824225


/home/mrandl/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[LibLinear]....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
optimization finished, #iter = 5000

Using -s 2 may be faster (also see FAQ)

Objective value = -79954.749924
nSV = 119197
0.82115


/home/mrandl/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[LibLinear]........................................................................................................................................................

In [ ]:
def publishResults(test_pred, file_name):
    with open('data/' + file_name, "w") as f:
        f.write("Id,Prediction\n")
        for pred, index in zip(test_pred, range(1, len(test_pred) +1)):
            f.write(str(index) + "," + str(pred) + "\n")

In [ ]:
with open('data/pickle/test_data.pkl', 'rb') as f:
    test_vecs = np.array(pickle.load(f))
test_vecs = (test_vecs - mean) / std
test_pred = train_and_pred(x, y, test_vecs)

In [ ]:
test_pred = [-1 if x <= 0.0 else 1 for x in test_pred]
publishResults(test_pred, "bert_lsvc.csv")

In [ ]:
test_pred